<a href="https://colab.research.google.com/github/mohammedradman1/DSE_Term_Project/blob/main/DSE_Term_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obtaining Data from Git Repository

Obtain the data from the project repository in Github

In [2]:
# obtaining the data from Git Repository
!wget https://github.com/mohammedradman1/DSE_Term_Project/blob/main/TeePublic_review.zip

--2023-12-25 18:41:32--  https://raw.githubusercontent.com/mohammedradman1/DSE_Term_Project/main/VIW_FNT.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25194738 (24M) [text/plain]
Saving to: ‘VIW_FNT.csv’

VIW_FNT.csv         100%[===================>]  24.03M   150MB/s    in 0.2s    

2023-12-25 18:41:33 (150 MB/s) - ‘VIW_FNT.csv’ saved [25194738/25194738]



In [ ]:
!unzip 'TeePublic_review.zip'

In [27]:
# Reading CSV file into pandas dataframe.
import pandas as pd
import numpy as np
df = pd.read_csv("VIW_FNT.csv") # reading the datafile
df.head()

,WHOREGION,FLUSEASON,HEMISPHERE,ITZ,COUNTRY_CODE,COUNTRY_AREA_TERRITORY,ISO_WEEKSTARTDATE,ISO_YEAR,ISO_WEEK,MMWR_WEEKSTARTDATE,...,PARAINFLUENZA,RHINO,RSV,OTHERRESPVIRUS,OTHER_RESPVIRUS_DETAILS,LAB_RESULT_COMMENT,WCR_COMMENT,ISO2,ISOYW,MMWRYW
0,AFR,NH,NH,FLU_NRT_AFR,DZA,Algeria,2021-11-29,2021,48,2021-11-28,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,DZ,202148,202148
1,WPR,YR,NH,FLU_SE_ASIA,KHM,Cambodia,2015-06-15,2015,25,2015-06-14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KH,201525,201524
2,SEAR,YR,NH,FLU_SE_ASIA,MMR,Myanmar,2023-03-13,2023,11,2023-03-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MM,202311,202311
3,EUR,NH,NH,FLU_CNT_ASIA,UZB,Uzbekistan,2017-02-13,2017,7,2017-02-12,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,UZ,201707,201707
4,EUR,NH,NH,FLU_SW_EUR,ALB,Albania,2023-04-24,2023,17,2023-04-23,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,AL,202317,202317


# Exploratory Data Analysis

In [5]:
# checking the shape of the data
df.shape

(150571, 49)

In [ ]:
# viewing a sample of the data
df.sample(5)

,WHOREGION,FLUSEASON,HEMISPHERE,ITZ,COUNTRY_CODE,COUNTRY_AREA_TERRITORY,ISO_WEEKSTARTDATE,ISO_YEAR,ISO_WEEK,MMWR_WEEKSTARTDATE,...,PARAINFLUENZA,RHINO,RSV,OTHERRESPVIRUS,OTHER_RESPVIRUS_DETAILS,LAB_RESULT_COMMENT,WCR_COMMENT,ISO2,ISOYW,MMWRYW
23919,AMR,NH,NH,FLU_NRT_AMR,CAN,Canada,2012-05-28,2012,22,2012-05-27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA,201222,201222
131661,EUR,NH,NH,FLU_EST_EUR,RUS,Russian Federation,2018-12-17,2018,51,2018-12-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RU,201851,201851
118765,WPR,YR,NH,FLU_SE_ASIA,PHL,Philippines,2017-11-27,2017,48,2017-11-26,...,0.0,0.0,0.0,0.0,None,NaN,NaN,PH,201748,201748
2810,AMR,YR,NH,FLU_CNT_AMC,GTM,Guatemala,2023-07-17,2023,29,2023-07-16,...,0.0,2.0,8.0,0.0,NaN,NaN,NaN,GT,202329,202329
117150,EUR,NH,NH,FLU_EST_EUR,SVK,Slovakia,2013-06-10,2013,24,2013-06-09,...,NaN,NaN,0.0,NaN,RSV,NaN,NaN,SK,201324,201324


In [ ]:
# Display the statistical analysis of numerical columns (mean, std, min, max, etc).
df.describe()

,ISO_YEAR,ISO_WEEK,MMWR_YEAR,MMWR_WEEK,SPEC_PROCESSED_NB,SPEC_RECEIVED_NB,AH1N12009,AH1,AH3,AH5,...,ADENO,BOCA,HUMAN_CORONA,METAPNEUMO,PARAINFLUENZA,RHINO,RSV,OTHERRESPVIRUS,ISOYW,MMWRYW
count,56419.000000,56419.000000,56419.000000,56419.000000,53602.000000,19491.000000,44109.000000,38870.000000,50116.000000,16984.000000,...,7242.000000,5336.000000,5136.000000,6586.000000,7293.000000,6173.000000,26262.000000,5945.000000,56418.000000,56418.000000
mean,2015.041032,26.160531,2015.039526,26.161541,460.983489,840.351342,15.053572,0.878107,11.329595,0.007183,...,6.786661,0.685907,4.174455,6.815518,9.370355,16.433177,15.770200,10.256182,201530.251427,201530.101776
std,6.402424,15.611284,6.403866,15.611054,3342.665627,5298.822857,166.107746,13.962122,106.447050,0.164063,...,21.707479,3.537126,24.313578,31.452069,34.939256,77.080739,94.315548,56.452488,640.199705,640.316665
min,1995.000000,1.000000,1995.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,199501.000000,199501.000000
25%,2011.000000,12.000000,2011.000000,12.000000,3.000000,11.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,201146.000000,201146.000000
50%,2016.000000,26.000000,2016.000000,26.000000,26.000000,38.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,201644.000000,201644.000000
75%,2020.000000,41.000000,2020.000000,41.000000,96.000000,116.000000,2.000000,0.000000,2.000000,0.000000,...,3.000000,0.000000,1.000000,2.000000,3.000000,4.000000,2.000000,0.000000,202039.000000,202039.000000
max,2023.000000,53.000000,2023.000000,53.000000,157960.000000,157960.000000,10575.000000,847.000000,12173.000000,14.000000,...,376.000000,102.000000,530.000000,723.000000,463.000000,1416.000000,3523.000000,704.000000,202350.000000,202350.000000


In [21]:
# checking for null values in columns
df.isnull().sum()

WHOREGION                       0
FLUSEASON                       0
HEMISPHERE                      0
ITZ                             0
COUNTRY_CODE                    0
COUNTRY_AREA_TERRITORY          0
ISO_WEEKSTARTDATE               0
ISO_YEAR                        0
ISO_WEEK                        0
MMWR_WEEKSTARTDATE              0
MMWR_YEAR                       0
MMWR_WEEK                       0
ORIGIN_SOURCE                   0
SPEC_PROCESSED_NB            7637
SPEC_RECEIVED_NB            98464
AH1N12009                   32964
AH1                         46991
AH3                         16878
AH5                        105192
AH7N9                      133389
ANOTSUBTYPED                20787
ANOTSUBTYPABLE             137894
AOTHER_SUBTYPE             108375
AOTHER_SUBTYPE_DETAILS     143663
INF_A                       12387
BVIC_2DEL                  143629
BVIC_3DEL                  144794
BVIC_NODEL                  39001
BVIC_DELUNK                143163
BYAM          

In [22]:
df['AH1'].value_counts()

0.0      98340
1.0       1622
2.0        769
3.0        486
4.0        307
         ...  
167.0        1
322.0        1
355.0        1
78.0         1
204.0        1
Name: AH1, Length: 235, dtype: int64